# Import required modules

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
from skimage import data
import skimage
from skimage.filters.thresholding import threshold_li,threshold_local,threshold_otsu
from skimage.morphology import erosion, dilation, opening, closing, white_tophat, remove_small_objects
from skimage.morphology import disk
from scipy import ndimage as ndi
import sys,os, glob

#import skimage.filters.median

#import skimage.segmentation as seg
#import skimage.filters as filters
#import skimage.draw as draw
#import skimage.color as color

#from skimage.filters.thresholding import _cross_entropy
#from skimage.morphology import black_tophat, skeletonize, convex_hull_image


#pip install nd2reader_required for nd2 file reading
from nd2reader import ND2Reader

import pandas as pd

c:\programdata\anaconda3\lib\site-packages\skimage\io\manage_plugins.py:23: UserWarning: Your installed pillow version is < 7.1.0. Several security issues (CVE-2020-11538, CVE-2020-10379, CVE-2020-10994, CVE-2020-10177) have been fixed in pillow 7.1.0 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


# Define parameters for image loading here:

In [2]:
# Common parameters for loading the image files of interest

# for 4 channels nd2 images: channel index order is 1-2-3-4, which is the order as ch405-ch488-ch560-ch647


# analysis done
num_of_ch = 4

# Replace protein/dna label name in the "xxx" below for the indicated channel index (make sure the order is correct)
# For example: ch_dict = {"dna":1,"rpa/rpa2":2,"mdc1":3,"pol2S5":4} if rpa/rpa2 was used for ch488
# Make sure the label name is the same as the factor/dna key used in the line below
ch_dict = {"dna":1,"Mdc1":2,"Fibirill":3,"Pol2Se":4}

# If use mdc1 as the factor for region segementation
# If use dna/dapi as the key channel for nucleus segmentation
factor_key =  "Mdc1"
dna_key = 'Pol2Se'  # temporary fix since DNA is not good
nucleoli_key="Fibirill"
pol2_key = 'Pol2Se'



# Replace the data directory in the ""; * is the final path level where images are located
# For example: data_save_folder = r"F:\XXX\AAA\BBB\*"
data_save_folder = r"D:\Analyzed_CellBio\Stam\cov1_Mdc1-488 Fibrill-568 PolS5-647\*"

data_files = [file for file in glob.glob(data_save_folder) if 'nd2' in file]
data_files

['D:\\Analyzed_CellBio\\Stam\\cov1_Mdc1-488 Fibrill-568 PolS5-647\\cov1_RPE1 Mdc1-488 Fibrill-568 PolS5-647_.nd2',
 'D:\\Analyzed_CellBio\\Stam\\cov1_Mdc1-488 Fibrill-568 PolS5-647\\cov1_RPE1 Mdc1-488 Fibrill-568 PolS5-647_001.nd2',
 'D:\\Analyzed_CellBio\\Stam\\cov1_Mdc1-488 Fibrill-568 PolS5-647\\cov1_RPE1 Mdc1-488 Fibrill-568 PolS5-647_002.nd2']

In [3]:
# bool setting for adjusting code indendation
_analyze_all_nd = True


# print progress
_verbose = True


# bad fovs to exclude
fov_to_exclude = [7-1, 19-1]



# The pixel size for excluding small 53BP_body;
# Replace *300* with other number desired or *0* if do not want to perform 53BP1 body foci exclusion
#small_53BP_size = 300
# other parameteres for image analysis


small_53BP_size = 0

h2ax_border_size_filter = 1

#nuclei_filter = 5000 # for 1004

nuclei_filter = 600

iqr_ratio = 2

std_ratio = 3

#coord_dist = 75
#cell_size =400


search_xylim = [500,1500]

# Analyze the effect of using different threshold for Pol2 

In [4]:
_measurement_all_fov_df = []

for data in data_files[1:2]:
    
    #measurement_for_all_fov = []
    
    data_name = data.split('\\')[-1]
    
    # load data of interest from preprocessed cell_info_dict
    #if data_name in cell_info_dict.keys():
    if _analyze_all_nd:
        if _verbose:
            print(f"-- Start analyzing the dataset of {data_name}")
        
        images = ND2Reader(data)
        num_of_planes = images.sizes["z"]
        num_of_fov = images.sizes["v"]
        
        
        # exclude the focal plane that are not focused
        fovs_all = list(range(num_of_fov))
        fov_of_interest = [ind for ind in fovs_all if ind not in fov_to_exclude]
        
    #####################################################################################
        # going through all fov of interest
        for _fov_id in fov_of_interest: 
        
            # Find the best focal plane using the m6T (the factor key) channel   
            image_fl = []
            image_std = []
            for i in range(num_of_planes):
                # going through all focal planes 
                image_array = np.array(images.get_frame_2D (v=_fov_id, c= ch_dict[factor_key], z=i))
                _fl=image_array.flatten()
                image_fl.append(_fl)
                image_std.append(np.std(_fl))
            best_plane_index = np.argmax(np.array(image_std))
            if _verbose:
                print(f"-- Analyzing the plane {best_plane_index+1} for the image {_fov_id+1} in this dataset")
                
            # load the best focal plane
            img_1_bf=np.array(images.get_frame_2D (v=_fov_id, c= 0, z=best_plane_index))
            img_2_bf=np.array(images.get_frame_2D (v=_fov_id, c= 1, z=best_plane_index))
            img_3_bf=np.array(images.get_frame_2D (v=_fov_id, c= 2, z=best_plane_index))
            img_4_bf=np.array(images.get_frame_2D (v=_fov_id, c= 3, z=best_plane_index))
            
            if _analyze_all_nd:
                # Update/Generate the img_dict
                #ch_img_dict={'1':crop_img_1,'2':crop_img_2,'3':crop_img_3,'4':crop_img_4}
                ch_img_dict={'1':img_1_bf,'2':img_2_bf,'3':img_3_bf,'4':img_4_bf}
                

                # Use Li_global_th and binary operations on the dna (POL2) channel to generate nuclei masks
                li_value =  threshold_otsu (ch_img_dict[str(ch_dict[dna_key])])
                nuclei_mask = ch_img_dict[str(ch_dict[dna_key])]>li_value
                
                erosion_factor_dna = 3
                eroded_nuclei_mask = dilation(nuclei_mask, disk(erosion_factor_dna))
                eroded_nuclei_mask = erosion(eroded_nuclei_mask, disk(erosion_factor_dna))
                
                eroded_nuclei_mask = ndi.binary_fill_holes(eroded_nuclei_mask)
                eroded_nuclei_mask = remove_small_objects(eroded_nuclei_mask, nuclei_filter,connectivity=1)
                smoothed_nuclei_mask = skimage.filters.median (eroded_nuclei_mask, disk(10))
                
                # Nuclei segmentation to get all valid nuclei of interest
                labeled_nuclei, num_of_nuclei = ndi.label(eroded_nuclei_mask)
                
                
                
                
                # Get global mdc mask (faster, less variation between cells as it is Ab staining)
                mdc_intensity = ch_img_dict[str(ch_dict[factor_key])]*smoothed_nuclei_mask
                mdc_intensity_filtered = np.array([i for i in mdc_intensity.flatten() if i >0])
                #mdc_iqr = (np.percentile(mdc_intensity_filtered,75)-np.percentile(mdc_intensity_filtered,25))
                # use std for threshold instead
                mdc_positive_th =np.mean(mdc_intensity_filtered) + np.std(mdc_intensity_filtered)*std_ratio
                #mdc_positive_th = np.percentile(mdc_intensity_filtered,75) + mdc_iqr*iqr_ratio
                mdc_mask = ch_img_dict[str(ch_dict[factor_key])]>mdc_positive_th
                mdc_mask = remove_small_objects(mdc_mask, 50,connectivity=1)
                
                
                # Get global fibrill mask (faster, less variation between cells as it is Ab staining)
                nuc_intensity = ch_img_dict[str(ch_dict[nucleoli_key])]*smoothed_nuclei_mask
                nuc_intensity_filtered = np.array([i for i in nuc_intensity.flatten() if i >0])
                nuc_positive_th =np.mean(nuc_intensity_filtered) + np.std(nuc_intensity_filtered)*3
                nuc_mask = ch_img_dict[str(ch_dict[nucleoli_key])]>nuc_positive_th
                nuc_mask = dilation(nuc_mask,disk(1))
                nuc_mask = erosion(nuc_mask,disk(1))
                nuc_mask = remove_small_objects(nuc_mask, 10,connectivity=1)
                
                # Get different mask for Pol2
                
                pol2_all_masks = []
                
                
                pol2_intensity = ch_img_dict[str(ch_dict[pol2_key])]*smoothed_nuclei_mask
                pol2_intensity_filtered = np.array([i for i in pol2_intensity.flatten() if i >0])
                # otsu for whole FOV
                pol2_otsu_th = threshold_otsu(ch_img_dict[str(ch_dict[pol2_key])])
                pol2_otsu_mask = ch_img_dict[str(ch_dict[pol2_key])]>pol2_otsu_th
                pol2_otsu_mask = erosion(smoothed_nuclei_mask, disk(10))*(pol2_otsu_mask==0)
                pol2_otsu_mask = remove_small_objects(pol2_otsu_mask, 10,connectivity=1)
                pol2_all_masks.append(pol2_otsu_mask)
                # otsu for whole cell
                pol2_otsu_cell_th = threshold_otsu(pol2_intensity)
                pol2_otsu_cell_mask = ch_img_dict[str(ch_dict[pol2_key])]>pol2_otsu_cell_th
                pol2_otsu_cell_mask = erosion(smoothed_nuclei_mask, disk(10))*(pol2_otsu_cell_mask==0)
                pol2_otsu_cell_mask = remove_small_objects(pol2_otsu_cell_mask, 10,connectivity=1)
                pol2_all_masks.append(pol2_otsu_cell_mask)
                # different percentile for th
                pol2_percentile_list = [10,20,30,40,50]
                for _percentile in pol2_percentile_list:
                    pol2_percentile_th = np.percentile(pol2_intensity_filtered, _percentile)
                    pol2_percentile_mask = ch_img_dict[str(ch_dict[pol2_key])]>pol2_percentile_th
                    pol2_percentile_mask = erosion(smoothed_nuclei_mask, disk(10))*(pol2_percentile_mask==0)
                    pol2_percentile_mask = remove_small_objects(pol2_percentile_mask, 10,connectivity=1)  
                    pol2_all_masks.append(pol2_percentile_mask)
                    
                # Prepare to analyze cell
                kept_nuclei = []
                for i in range(num_of_nuclei):
                    cand_nucleus = labeled_nuclei == i+1
                    cand_nucleus[cand_nucleus>0]=1
                    cand_nucleus = np.array(cand_nucleus)
                    
                    # find the labeled nuclei close enough to the given coord (with 50 pixel)
                    region = skimage.measure.regionprops (skimage.measure.label(cand_nucleus))[0]
                    if region.centroid[0] > search_xylim[0] and region.centroid[0] < search_xylim[1]:
                        if region.centroid[1] > search_xylim[0] and region.centroid[1] < search_xylim[1]:
                            kept_nuclei.append(cand_nucleus)
                ##########################################################################    
                
                
                # Analyze each valid nuclei   
                for _cell_id, kept_nucleus in enumerate(kept_nuclei):
                    if _verbose:
                        print(f'-- Analyzing cell {_cell_id+1} in fov {_fov_id+1}.')
                        
                    nuclei_to_measure = kept_nucleus
                
                    # check if this cell has mdc-chr incorporation
                    mdc_chr_mask = np.logical_and(mdc_mask,nuclei_to_measure)
        
                    # get fibrill-positive mask            
                    nuc_chr_mask = np.logical_and(nuc_mask,nuclei_to_measure)
                    
                    # exlcude bad fib stained cells
                    if np.sum (nuc_chr_mask) > 100:

                        # get the normalized area for fibrill within the pol2 mask
                    
                        _measurement_each_cell = pd.DataFrame()
                        _df_columns = ['assigned_fib_global_otsu','fib_in_po2_global_otsu',
                                   'assigned_fib_cell_otsu','fib_in_po2_cell_otsu',
                                   'assigned_fib_p10','fib_in_po2_p10',
                                   'assigned_fib_p20','fib_in_po2_p20',
                                   'assigned_fib_p30','fib_in_po2_p30',
                                   'assigned_fib_p40','fib_in_po2_p40',
                                   'assigned_fib_p50','fib_in_po2_p50',]
                    
                        _measurement_list = []
                        for _pol2_mask in pol2_all_masks:
                            fib_in_pol2 = nuc_chr_mask * _pol2_mask
                            # this measures the ratio of fib were assigned to Pol2 mask relative to all fib:
                            # if Pol2 mask too strignent, many fib will be left outside, leading to low value
                            fib_ratio_in_pol2 = np.sum(fib_in_pol2)/np.sum(nuc_chr_mask)
                            _measurement_list.append(fib_ratio_in_pol2)
                        
                        # this measures the ratio of fib area in the pol2 area: 
                        # if Pol2 mask too loose, the Pol2-area occupied by fib would become low, leading to low value
                            fib_area_ratio_in_pol2 = np.sum(fib_in_pol2)/np.sum(_pol2_mask*nuclei_to_measure)
                            _measurement_list.append(fib_area_ratio_in_pol2)
                    
                        for _measurement, _df_col in zip(_measurement_list, _df_columns):
                            _measurement_each_cell[_df_col] = [_measurement]
                    
                        if np.sum(mdc_chr_mask)<30:
                            _measurement_each_cell.insert(0,'mdc positive chr','No')
                        else:
                            _measurement_each_cell.insert(0,'mdc positive chr','Yes')
                        
                        _measurement_each_cell.insert(0,'cell id',_cell_id+1)
                        _measurement_each_cell.insert(0,'fov id',_fov_id+1)
                    
                    
                        if len(_measurement_all_fov_df) == 0:
                            _measurement_all_fov_df = _measurement_each_cell
                        else:
                            _measurement_all_fov_df = pd.concat([_measurement_all_fov_df,_measurement_each_cell])
                    


-- Start analyzing the dataset of cov1_RPE1 Mdc1-488 Fibrill-568 PolS5-647_001.nd2
-- Analyzing the plane 6 for the image 1 in this dataset
-- Analyzing cell 1 in fov 1.
-- Analyzing cell 2 in fov 1.
-- Analyzing cell 3 in fov 1.
-- Analyzing cell 4 in fov 1.
-- Analyzing cell 5 in fov 1.
-- Analyzing cell 6 in fov 1.
-- Analyzing cell 7 in fov 1.
-- Analyzing cell 8 in fov 1.
-- Analyzing cell 9 in fov 1.
-- Analyzing the plane 6 for the image 2 in this dataset
-- Analyzing cell 1 in fov 2.
-- Analyzing cell 2 in fov 2.
-- Analyzing cell 3 in fov 2.
-- Analyzing cell 4 in fov 2.
-- Analyzing cell 5 in fov 2.
-- Analyzing cell 6 in fov 2.
-- Analyzing cell 7 in fov 2.
-- Analyzing cell 8 in fov 2.
-- Analyzing cell 9 in fov 2.
-- Analyzing cell 10 in fov 2.
-- Analyzing the plane 6 for the image 3 in this dataset
-- Analyzing cell 1 in fov 3.
-- Analyzing cell 2 in fov 3.
-- Analyzing cell 3 in fov 3.
-- Analyzing cell 4 in fov 3.
-- Analyzing cell 5 in fov 3.
-- Analyzing cell 6 in fo

C:\Users\Shiwei\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:171: RuntimeWarning: invalid value encountered in long_scalars


-- Analyzing cell 7 in fov 5.
-- Analyzing cell 8 in fov 5.
-- Analyzing cell 9 in fov 5.
-- Analyzing cell 10 in fov 5.
-- Analyzing cell 11 in fov 5.
-- Analyzing cell 12 in fov 5.
-- Analyzing cell 13 in fov 5.
-- Analyzing cell 14 in fov 5.
-- Analyzing the plane 6 for the image 6 in this dataset
-- Analyzing cell 1 in fov 6.
-- Analyzing cell 2 in fov 6.
-- Analyzing cell 3 in fov 6.
-- Analyzing cell 4 in fov 6.
-- Analyzing cell 5 in fov 6.
-- Analyzing cell 6 in fov 6.
-- Analyzing cell 7 in fov 6.
-- Analyzing cell 8 in fov 6.
-- Analyzing cell 9 in fov 6.
-- Analyzing cell 10 in fov 6.
-- Analyzing the plane 6 for the image 8 in this dataset
-- Analyzing cell 1 in fov 8.
-- Analyzing cell 2 in fov 8.
-- Analyzing the plane 5 for the image 9 in this dataset
-- Analyzing cell 1 in fov 9.
-- Analyzing cell 2 in fov 9.
-- Analyzing cell 3 in fov 9.
-- Analyzing cell 4 in fov 9.
-- Analyzing cell 5 in fov 9.
-- Analyzing cell 6 in fov 9.
-- Analyzing the plane 6 for the image 10 i

In [5]:
_measurement_all_fov_df

,fov id,cell id,mdc positive chr,assigned_fib_global_otsu,fib_in_po2_global_otsu,assigned_fib_cell_otsu,fib_in_po2_cell_otsu,assigned_fib_p10,fib_in_po2_p10,assigned_fib_p20,fib_in_po2_p20,assigned_fib_p30,fib_in_po2_p30,assigned_fib_p40,fib_in_po2_p40,assigned_fib_p50,fib_in_po2_p50
0,1,1,No,0.546778,0.224403,0.365904,0.249292,0.311850,0.235849,0.546778,0.224403,0.663202,0.193921,0.750520,0.169723,0.810811,0.149368
0,1,2,No,0.415361,0.196151,0.250784,0.169312,0.233542,0.167040,0.415361,0.196151,0.550157,0.204545,0.658307,0.198207,0.736677,0.178707
0,1,3,No,0.225722,0.367521,0.173228,0.478261,0.165354,0.488372,0.225722,0.367521,0.291339,0.338415,0.346457,0.285714,0.409449,0.231454
0,1,4,No,0.502674,0.057528,0.352941,0.057996,0.336898,0.059378,0.502674,0.057528,0.614973,0.057673,0.679144,0.053227,0.700535,0.046686
0,1,5,No,0.902200,0.192188,0.850856,0.228496,0.833741,0.231500,0.902200,0.192188,0.933985,0.146641,0.960880,0.105080,0.982885,0.080416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,21,2,No,0.455142,0.478161,0.306346,0.606061,0.321663,0.544444,0.487965,0.454175,0.595186,0.374656,0.663020,0.286389,0.754923,0.231233
0,21,3,No,0.133475,0.360000,0.055085,0.382353,0.063559,0.389610,0.182203,0.316176,0.358051,0.285956,0.468220,0.224138,0.574153,0.184983
0,21,4,Yes,0.333124,0.578431,0.226474,0.584142,0.244040,0.573746,0.364492,0.555449,0.465496,0.498991,0.560226,0.420235,0.653074,0.349329
0,21,5,Yes,1.000000,0.036676,1.000000,0.057244,1.000000,0.052828,1.000000,0.032983,1.000000,0.022856,1.000000,0.017093,1.000000,0.012581


In [6]:
excel_save_folder = r"D:\Analyzed_CellBio\Stam\cov1_Mdc1-488 Fibrill-568 PolS5-647"

_measurement_all_fov_df.to_excel(excel_save_folder + os.sep+ "Fibrill_PolS5_exp1_Fib_3std.xlsx", index=False)

In [7]:
_measurement_all_fov_df["score_global_otsu"] = _measurement_all_fov_df["assigned_fib_global_otsu"]*_measurement_all_fov_df["fib_in_po2_global_otsu"]


In [8]:
_measurement_all_fov_df["score_cell_otsu"]  = _measurement_all_fov_df["assigned_fib_cell_otsu"]*_measurement_all_fov_df["fib_in_po2_cell_otsu"]

_measurement_all_fov_df["score_p10"]  = _measurement_all_fov_df["assigned_fib_p10"]*_measurement_all_fov_df["fib_in_po2_p10"]

_measurement_all_fov_df["score_p20"]  = _measurement_all_fov_df["assigned_fib_p20"]*_measurement_all_fov_df["fib_in_po2_p20"]

_measurement_all_fov_df["score_p30"]  = _measurement_all_fov_df["assigned_fib_p30"]*_measurement_all_fov_df["fib_in_po2_p30"]

_measurement_all_fov_df["score_p40"]  = _measurement_all_fov_df["assigned_fib_p40"]*_measurement_all_fov_df["fib_in_po2_p40"]

_measurement_all_fov_df["score_p50"]  = _measurement_all_fov_df["assigned_fib_p50"]*_measurement_all_fov_df["fib_in_po2_p50"]

In [9]:
_measurement_all_fov_df.groupby(by=['mdc positive chr']).mean()

,fov id,cell id,assigned_fib_global_otsu,fib_in_po2_global_otsu,assigned_fib_cell_otsu,fib_in_po2_cell_otsu,assigned_fib_p10,fib_in_po2_p10,assigned_fib_p20,fib_in_po2_p20,...,fib_in_po2_p40,assigned_fib_p50,fib_in_po2_p50,score_global_otsu,score_cell_otsu,score_p10,score_p20,score_p30,score_p40,score_p50
mdc positive chr,,,,,,,,,,,,,,,,,,,,,
No,10.792079,5.831683,0.667159,0.229578,0.549167,0.276067,0.552229,0.272530,0.673804,0.226446,...,0.163677,0.825144,0.136937,0.152095,0.153931,0.152310,0.151656,0.140320,0.125361,0.109806
Yes,12.600000,4.942857,0.548574,0.267005,0.452944,0.330944,0.456559,0.331884,0.553479,0.263551,...,0.212600,0.734348,0.184567,0.143998,0.159767,0.159082,0.144101,0.138293,0.133352,0.126020


In [10]:
_measurement_all_fov_df.to_excel(excel_save_folder + os.sep+ "Fibrill_PolS5_exp1_Fib_3std_score.xlsx", index=False)